(27.5, 4*sqrt(2), 12.5, 13, 17)

In [ ]:
from sympy import *


r1=27.5
r2=12.5
r3=13
r4=17
r5=4*sqrt(2)

r1,r5,r2,r3,r4
tt1=0
tt2=atan(10/7.5)
tt3=atan(5/12)
tt4=atan(15/-8)
tt5=atan(4/-4)
float(tt1/pi*180),float(tt2/pi*180),float(tt3/pi*180),float(tt4/pi*180),float(tt5/pi*180)


w1=0
w4=1

w2,w3=S('w2,w3')
q1=r2*w2*cos(tt2)+r3*w3*cos(tt3)-r1*w1*cos(tt1)-r4*w4*cos(tt4)
q2=r2*w2*sin(tt2)+r3*w3*sin(tt3)-r1*w1*sin(tt1)-r4*w4*sin(tt4)


solve([q1,q2],[w2,w3])



{w2: -2.66666666666667, w3: 2.33333333333333}

In [18]:
import numpy as np

# --- 1. 定义已知条件 ---
# 各点坐标 (单位：任意长度单位，这里假设为 cm)
O2 = np.array([0.0, 0.0])
O4 = np.array([27.5, 0.0])
A = np.array([7.5, 10.0])
B = np.array([19.5, 15.0])
C = np.array([3.5, 14.0]) # C点用于后续分析，核心计算暂时不用

# 连杆4的已知运动学参数
omega4 = 1.0   # rad/s
alpha4 = -2.0  # rad/s^2

print("--- 输入参数 ---")
print(f"O2 = {O2}, O4 = {O4}, A = {A}, B = {B}")
print(f"ω4 = {omega4} rad/s")
print(f"α4 = {alpha4} rad/s^2")
print("-" * 20 + "\n")


# --- 2. 位置分析 (Position Analysis) ---
# 计算连杆矢量 r_O2A (r2), r_AB (r3), r_O4B (r4)
# r1是地杆 O2->O4
r2 = A - O2
r3 = B - A
r4 = B - O4
r1 = O4 - O2

# 验证矢量环路是否闭合: r2 + r3 = r1 + r4
if not np.allclose(r2 + r3, r1 + r4):
    raise ValueError("几何结构不闭合，请检查输入坐标。")

# 计算各连杆的长度
L2 = np.linalg.norm(r2)
L3 = np.linalg.norm(r3)
L4 = np.linalg.norm(r4)

# 计算各连杆的瞬时角度 (使用arctan2确保象限正确)
theta2 = np.arctan2(r2[1], r2[0])
theta3 = np.arctan2(r3[1], r3[0])
theta4 = np.arctan2(r4[1], r4[0])

print("--- 位置分析结果 ---")
print(f"连杆矢量 r2 = {r2}, 长度 L2 = {L2:.4f}, 角度 θ2 = {np.degrees(theta2):.4f}°")
print(f"连杆矢量 r3 = {r3}, 长度 L3 = {L3:.4f}, 角度 θ3 = {np.degrees(theta3):.4f}°")
print(f"连杆矢量 r4 = {r4}, 长度 L4 = {L4:.4f}, 角度 θ4 = {np.degrees(theta4):.4f}°")
print("-" * 20 + "\n")


# --- 3. 速度分析 (Velocity Analysis) ---
# 速度方程: ω2 x r2 + ω3 x r3 = ω4 x r4
# 展开为2D线性方程组: M * [ω2, ω3]' = V
# M = [[-r2_y, -r3_y], [r2_x, r3_x]]
# V = [-r4_y*ω4, r4_x*ω4]'

# 构建系数矩阵 M
M = np.array([
    [-r2[1], -r3[1]],
    [r2[0],  r3[0]]
])

# 构建右侧矢量 V
V = np.array([
    -r4[1] * omega4,
    r4[0] * omega4
])

# 求解 [ω2, ω3]
try:
    omega_solution = np.linalg.solve(M, V)
    omega2 = omega_solution[0]
    omega3 = omega_solution[1]
    
    print("--- 速度分析结果 ---")
    print(f"角速度 ω2 = {omega2:.4f} rad/s")
    print(f"角速度 ω3 = {omega3:.4f} rad/s")
    print("-" * 20 + "\n")

except np.linalg.LinAlgError:
    print("速度分析失败：矩阵为奇异矩阵，机构可能处于死点位置。")
    # 如果无法求解，则退出
    exit()


# --- 4. 加速度分析 (Acceleration Analysis) ---
# 加速度方程: α2xr2 + α3xr3 = α4xr4 - ω2^2*r2 - ω3^2*r3 + ω4^2*r4
# 同样展开为2D线性方程组: M * [α2, α3]' = A_rhs
# M 矩阵与速度分析中的相同
# A_rhs = (α4xr4) - (ω2^2*r2) - (ω3^2*r3) + (ω4^2*r4)

# 计算 α4 x r4 矢量
alpha4_cross_r4 = np.array([-r4[1] * alpha4, r4[0] * alpha4])

# 构建右侧矢量 A_rhs
A_rhs = alpha4_cross_r4 - (omega2**2 * r2) - (omega3**2 * r3) + (omega4**2 * r4)

# 求解 [α2, α3]
try:
    alpha_solution = np.linalg.solve(M, A_rhs)
    alpha2 = alpha_solution[0]
    alpha3 = alpha_solution[1]

    print("--- 加速度分析结果 ---")
    print(f"角加速度 α2 = {alpha2:.4f} rad/s^2")
    print(f"角加速度 α3 = {alpha3:.4f} rad/s^2")
    print("-" * 20 + "\n")

except np.linalg.LinAlgError:
    print("加速度分析失败：矩阵为奇异矩阵。")

--- 输入参数 ---
O2 = [0. 0.], O4 = [27.5  0. ], A = [ 7.5 10. ], B = [19.5 15. ]
ω4 = 1.0 rad/s
α4 = -2.0 rad/s^2
--------------------

--- 位置分析结果 ---
连杆矢量 r2 = [ 7.5 10. ], 长度 L2 = 12.5000, 角度 θ2 = 53.1301°
连杆矢量 r3 = [12.  5.], 长度 L3 = 13.0000, 角度 θ3 = 22.6199°
连杆矢量 r4 = [-8. 15.], 长度 L4 = 17.0000, 角度 θ4 = 118.0725°
--------------------

--- 速度分析结果 ---
角速度 ω2 = 2.6667 rad/s
角速度 ω3 = -2.3333 rad/s
--------------------

--- 加速度分析结果 ---
角加速度 α2 = 18.1414 rad/s^2
角加速度 α3 = -16.9495 rad/s^2
--------------------



In [19]:
import numpy as np

# --- 1. 定义已知条件 ---
# 各点坐标
O2 = np.array([0.0, 0.0])
O4 = np.array([27.5, 0.0])
A = np.array([7.5, 10.0])
B = np.array([19.5, 15.0])

# 连杆4的已知运动学参数
omega4 = 1.0   # rad/s
alpha4 = -2.0  # rad/s^2

# --- 2. 位置分析 ---
# 从坐标计算连杆的长度 (L) 和角度 (theta)
L1 = np.linalg.norm(O4 - O2)
L2 = np.linalg.norm(A - O2)
L3 = np.linalg.norm(B - A)
L4 = np.linalg.norm(B - O4)

r2_vec = A - O2
r3_vec = B - A
r4_vec = B - O4
theta2 = np.arctan2(r2_vec[1], r2_vec[0])
theta3 = np.arctan2(r3_vec[1], r3_vec[0])
theta4 = np.arctan2(r4_vec[1], r4_vec[0])

print("--- 位置分析结果 ---")
print(f"L2 = {L2:.4f}, θ2 = {np.degrees(theta2):.4f}°")
print(f"L3 = {L3:.4f}, θ3 = {np.degrees(theta3):.4f}°")
print(f"L4 = {L4:.4f}, θ4 = {np.degrees(theta4):.4f}°")
print("-" * 20 + "\n")


# --- 3. 速度分析 (Compact Form) ---
# 求解线性方程组: M * [ω2, ω3]' = V
# M 和 V 来自于速度方程的 i-轴 和 j-轴 分量

# 构建系数矩阵 M
M = np.array([
    [-L2 * np.sin(theta2), -L3 * np.sin(theta3)],  # i-轴方程的系数
    [ L2 * np.cos(theta2),  L3 * np.cos(theta3)]   # j-轴方程的系数
])

# 构建右侧矢量 V
V = np.array([
    -L4 * omega4 * np.sin(theta4), # i-轴方程的右侧
    L4 * omega4 * np.cos(theta4)  # j-轴方程的右侧
])

# 求解 [ω2, ω3]
omega_solution = np.linalg.solve(M, V)
omega2 = omega_solution[0]
omega3 = omega_solution[1]

print("--- 速度分析结果 ---")
print(f"角速度 ω2 = {omega2:.4f} rad/s")
print(f"角速度 ω3 = {omega3:.4f} rad/s")
print("-" * 20 + "\n")


# --- 4. 加速度分析 (Compact Form) ---
# 求解线性方程组: M * [α2, α3]' = A_rhs
# M 矩阵与速度分析中的相同
# A_rhs 来自于加速度方程的 i-轴 和 j-轴 分量

# 构建右侧矢量 A_rhs 的 i-轴分量 (x-component)
A_rhs_i = (L2 * omega2**2 * np.cos(theta2) + 
           L3 * omega3**2 * np.cos(theta3) - 
           L4 * alpha4 * np.sin(theta4) - 
           L4 * omega4**2 * np.cos(theta4))

# 构建右侧矢量 A_rhs 的 j-轴分量 (y-component)
A_rhs_j = (L2 * omega2**2 * np.sin(theta2) + 
           L3 * omega3**2 * np.sin(theta3) + 
           L4 * alpha4 * np.cos(theta4) - 
           L4 * omega4**2 * np.sin(theta4))

A_rhs = np.array([A_rhs_i, A_rhs_j])

# 求解 [α2, α3]
alpha_solution = np.linalg.solve(M, A_rhs)
alpha2 = alpha_solution[0]
alpha3 = alpha_solution[1]

print("--- 加速度分析结果 ---")
print(f"角加速度 α2 = {alpha2:.4f} rad/s^2")
print(f"角加速度 α3 = {alpha3:.4f} rad/s^2")
print("-" * 20 + "\n")

--- 位置分析结果 ---
L2 = 12.5000, θ2 = 53.1301°
L3 = 13.0000, θ3 = 22.6199°
L4 = 17.0000, θ4 = 118.0725°
--------------------

--- 速度分析结果 ---
角速度 ω2 = 2.6667 rad/s
角速度 ω3 = -2.3333 rad/s
--------------------

--- 加速度分析结果 ---
角加速度 α2 = -28.8081 rad/s^2
角加速度 α3 = 26.2828 rad/s^2
--------------------



In [26]:
import numpy as np

# --- 1. 定义已知条件 ---
# 各点坐标 (单位：cm)
O2 = np.array([0.0, 0.0])
O4 = np.array([27.5, 0.0])
A = np.array([7.5, 10.0])
B = np.array([19.5, 15.0])
C = np.array([3.5, 14.0])

# 连杆4的已知运动学参数
omega4 = 1.0   # rad/s
alpha4 = -2.0  # rad/s^2

# --- 2. 位置分析 ---
# 计算连杆矢量
r2 = A - O2
r3 = B - A
r4 = B - O4

# 计算各连杆的长度和角度
L2 = np.linalg.norm(r2)
L3 = np.linalg.norm(r3)
L4 = np.linalg.norm(r4)
theta2 = np.arctan2(r2[1], r2[0])
theta3 = np.arctan2(r3[1], r3[0])
theta4 = np.arctan2(r4[1], r4[0])

# --- 3. 速度与加速度分析 (求解 ω2, ω3, α2, α3) ---
# 构建并求解速度方程组
M = np.array([
    [-L2 * np.sin(theta2), -L3 * np.sin(theta3)],
    [ L2 * np.cos(theta2),  L3 * np.cos(theta3)]
])
V = np.array([
    -L4 * omega4 * np.sin(theta4),
     L4 * omega4 * np.cos(theta4)
])
omega_solution = np.linalg.solve(M, V)
omega2 = omega_solution[0]
omega3 = omega_solution[1]

# 构建并求解加速度方程组
A_rhs_i = (L2 * omega2**2 * np.cos(theta2) + 
           L3 * omega3**2 * np.cos(theta3) - 
           L4 * alpha4 * np.sin(theta4) - 
           L4 * omega4**2 * np.cos(theta4))
A_rhs_j = (L2 * omega2**2 * np.sin(theta2) + 
           L3 * omega3**2 * np.sin(theta3) + 
           L4 * alpha4 * np.cos(theta4) - 
           L4 * omega4**2 * np.sin(theta4))
A_rhs = np.array([A_rhs_i, A_rhs_j])
alpha_solution = np.linalg.solve(M, A_rhs)
alpha2 = alpha_solution[0]
alpha3 = alpha_solution[1]
print(alpha2,alpha3)
# ------------------------------------------------------------------
# --- 4. 计算 C 点的运动学特性 (rc, vc, ac) ---
# ------------------------------------------------------------------

print("--- C点运动分析 ---")

# --- 位置 rc ---
# 计算从A到C的矢量 r_C/A (该矢量与连杆3固连)
r_C_from_A = C - A
# C点的位置矢量 rc (从原点O2出发)
rc = r2 + r_C_from_A
print(f"位置矢量 rc = [{rc[0]:.4f}, {rc[1]:.4f}] cm  (与输入C点坐标一致)")

# --- 速度 vc ---
# 定义一个辅助函数用于计算2D中的叉乘 w x r
def cross_product_2d(omega, r_vector):
    # omega (w) 是一个标量 (z方向), r_vector 是一个2D矢量 [rx, ry]
    # w k x (rx i + ry j) = -w*ry i + w*rx j
    return np.array([-omega * r_vector[1], omega * r_vector[0]])

# 计算A点的速度 va = w2 x r2
va = cross_product_2d(omega2, r2)
# 计算C相对于A的速度 vc/a = w3 x r_C/A
vc_from_a = cross_product_2d(omega3, r_C_from_A)
# C点的绝对速度 vc = va + vc/a
vc = va + vc_from_a
vc_magnitude = np.linalg.norm(vc)

print(f"速度矢量 vc = [{vc[0]:.4f}, {vc[1]:.4f}] cm/s")
print(f"速度大小 |vc| = {vc_magnitude:.4f} cm/s")

# --- 加速度 ac ---
# 计算A点的加速度 aa = α2 x r2 - ω2^2 * r2
aa_tangential = cross_product_2d(alpha2, r2)
aa_normal = -omega2**2 * r2
aa = aa_tangential + aa_normal

# 计算C相对于A的加速度 ac/a = α3 x r_C/A - ω3^2 * r_C/A
ac_from_a_tangential = cross_product_2d(alpha3, r_C_from_A)
ac_from_a_normal = -omega3**2 * r_C_from_A
ac_from_a = ac_from_a_tangential + ac_from_a_normal

# C点的绝对加速度 ac = aa + ac/a
ac = aa + ac_from_a
ac_magnitude = np.linalg.norm(ac)

print(f"加速度矢量 ac = [{ac[0]:.4f}, {ac[1]:.4f}] cm/s^2")
print(f"加速度大小 |ac| = {ac_magnitude:.4f} cm/s^2")
print("-" * 20 + "\n")

-28.808080808080813 26.282828282828287
--- C点运动分析 ---
位置矢量 rc = [3.5000, 14.0000] cm  (与输入C点坐标一致)
速度矢量 vc = [-17.3333, 29.3333] cm/s
速度大小 |vc| = 34.0718 cm/s
加速度矢量 ac = [151.3939, -414.0808] cm/s^2
加速度大小 |ac| = 440.8889 cm/s^2
--------------------

